# 用蒙特卡洛方法(Monte Carlo method)求解某个策略下每个状态的state value

## 轮盘赌采样
A被选择的概率是0.3，B被选择的概率是0.6，C被选择的概率是0.1.
<br>从中按概率选取一个

In [10]:
import random
probabilities = {'A': 0.3, 'B': 0.6, 'C': 0.1}
r = random.random()  # 生成[0,1)的随机数，比如0.45
temp=0
for i in probabilities:
    temp += probabilities[i]
    if temp >= r:
        print(i)
        break

B


## 问题描述
如图对应《动手学强化学习》第三章Page26,求出某个策略下每个状态s的state value

<img src="./picture1_2.png" alt="插入图片哈哈" width="50%">

## 根据上图创建环境

In [11]:
import numpy as np
S = ["s1", "s2", "s3", "s4", "s5"]  # 状态集合
A = ["保持s1", "前往s1", "前往s2", "前往s3", "前往s4", "前往s5", "概率前往"]  # 动作集合
# 状态转移函数 , 这里的前往是确定性的状态转移，概率前往是随机选状态转移，对应赵书Page3
P = {"s1-保持s1-s1": 1.0,
    "s1-前往s2-s2": 1.0,
    "s2-前往s1-s1": 1.0,
    "s2-前往s3-s3": 1.0,
    "s3-前往s4-s4": 1.0,
    "s3-前往s5-s5": 1.0,
    "s4-前往s5-s5": 1.0,
    "s4-概率前往-s2": 0.2,
    "s4-概率前往-s3": 0.4,
    "s4-概率前往-s4": 0.4}
# 奖励函数
R = {"s1-保持s1": -1,
    "s1-前往s2": 0,
    "s2-前往s1": -1,
    "s2-前往s3": -2,
    "s3-前往s4": -2,
    "s3-前往s5": 0,
    "s4-前往s5": 10,
    "s4-概率前往": 1}
gamma = 0.5  # 折扣因子
MDP = (S, A, P, R, gamma)

## Policy1,和1-2中Policy1相同

In [12]:
# 策略1,随机策略
Pi_1 = {"s1-保持s1": 0.5,
        "s1-前往s2": 0.5,
        "s2-前往s1": 0.5,
        "s2-前往s3": 0.5,
        "s3-前往s4": 0.5,
        "s3-前往s5": 0.5,
        "s4-前往s5": 0.5,
        "s4-概率前往": 0.5}

## 蒙特卡罗方法求解该则策略下的state value

In [13]:
def join(str1, str2):
    return str1 + '-' + str2

In [14]:
def sample(MDP, Pi, timestep_max, number):
    ''' 采样函数,策略Pi,限制最长时间步timestep_max,number表示总共采样多少个eposides '''
    S, A, P, R, gamma = MDP
    episodes = []
    for _ in range(number):
        episode = [] #每个eposide的内容就是[(s, a, r, s_next)，(s, a, r, s_next)...]
        timestep = 0
        s = S[np.random.randint(4)]  # 随机选择一个除s5以外的状态s作为起点
        # 当前状态为终止状态或者时间步太长时,一次采样结束
        while s != "s5" and timestep <= timestep_max:
            timestep += 1
            rand, temp = np.random.rand(), 0
            # 在状态s下根据策略选择动作
            for a_opt in A:
                temp += Pi.get(join(s, a_opt),0) #组合一个state和action,state前面已经选了，这里要选一个action,如果这个策略存在则就是它的概率，如果不存在就是0，后面加上0也没说
                if temp > rand:                  #按概率选取pi
                    a = a_opt                    #确定这个action
                    r = R.get(join(s, a), 0)     #确定这个state,action获得的reward
                    break

            rand, temp = np.random.rand(), 0
            # 根据状态转移概率得到下一个状态s_next
            for s_opt in S:
                temp += P.get(join(join(s, a), s_opt), 0)
                if temp > rand:                  #根据概率，s,a->s_next,s,a已经确定，根据概率到达s_next
                    s_next = s_opt               #确定这个s_next
                    break
            episode.append((s, a, r, s_next))    #把（s,a,r,s_next）元组放入序列中
            s = s_next  # s_next变成当前状态,开始接下来的循环
        episodes.append(episode)
    return episodes

# 采样5次,每个序列最长不超过20步
episodes = sample(MDP, Pi_1, 20, 5)
print('第一条序列\n', episodes[0])
print('第二条序列\n', episodes[1])
print('第五条序列\n', episodes[4])

第一条序列
 [('s3', '前往s5', 0, 's5')]
第二条序列
 [('s3', '前往s5', 0, 's5')]
第五条序列
 [('s2', '前往s1', -1, 's1'), ('s1', '前往s2', 0, 's2'), ('s2', '前往s1', -1, 's1'), ('s1', '保持s1', -1, 's1'), ('s1', '保持s1', -1, 's1'), ('s1', '保持s1', -1, 's1'), ('s1', '前往s2', 0, 's2'), ('s2', '前往s3', -2, 's3'), ('s3', '前往s4', -2, 's4'), ('s4', '前往s5', 10, 's5')]


In [15]:
# 对所有采样序列计算所有状态的价值
def MC(episodes, V, N, gamma):
    for episode in episodes:
        G = 0
        for i in range(len(episode) - 1, -1, -1):  #一个序列从后往前计算
            (s, a, r, s_next) = episode[i]
            G = r + gamma * G
            N[s] = N[s] + 1                  #这个state访问次数+1
            V[s] = V[s] + (G - V[s]) / N[s]  #更新这个state value

timestep_max = 20
# 采样1000次,可以自行修改
episodes = sample(MDP, Pi_1, timestep_max, 2000)
gamma = 0.5
V = {"s1": 0, "s2": 0, "s3": 0, "s4": 0, "s5": 0}   #表示初始化每个状态的state value
N = {"s1": 0, "s2": 0, "s3": 0, "s4": 0, "s5": 0}   #表示初始化每个状态的访问次数
MC(episodes, V, N, gamma)
print("使用蒙特卡洛方法计算MDP的状态价值state value为\n", V)

使用蒙特卡洛方法计算MDP的状态价值state value为
 {'s1': -1.2173046676569945, 's2': -1.6863598535054698, 's3': 0.5163934921636807, 's4': 6.093268018889927, 's5': 0}
